In [1]:
local = {
    "AzureWebJobsStorage": "DefaultEndpointsProtocol=https;AccountName=fiapstorageaccount;AccountKey=kDtweiQcx7bAJMzo44AE5E054OPRFbzIx/O4mYZyOVQRjjWGy5nfi7gafyIk/vTRQsxCz6S/X+3y+AStxfbHWg==;EndpointSuffix=core.windows.net",
    "FUNCTIONS_WORKER_RUNTIME": "python",
    "COSMOS_NORMAL_ENDPOINT": "https://fiap-normal-database.documents.azure.com:443/",
    "COSMOS_NORMAL_PRIMARYKEY": "FpUhmIEBc4hSW2rLsm1xUkrdhdHpPhLGj1uRWFMx2BHzBrcJPr3oRbSDdIFFFQKqZGk4Nloclq2SACDbYIk3BQ==",
    "COSMOS_NORMAL_DATABASE": "musics",
    "COSMOS_GRAPH_ENDPOINT": "https://fiapgraphdatabase.documents.azure.com:443/",
    "COSMOS_GRAPH_PRIMARYKEY": "u7c8yDpUBourU636HhHU3WnJJhMz0mOVk1GzKf6fqfpEsudzgUegTZ3yby6Arbzq2OGdKzcwENm4ACDb2jlygw==",
    "COSMOS_GRAPH_DATABASE": "music_correlations",
    "SPOTIPY_CLIENT_ID":"eb1060ace869470199dfb41aaecca2ab",
    "SPOTIPY_CLIENT_SECRET":"42c96b477c534dbbad9d3e300c64353b",
    "SPOTIPY_REDIRECT_URI": "http://localhost:7071/api/callback_spotify"
  }

import os

for k,v in local.items():
    os.environ[k]=v

In [19]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials

# Set up the Spotify API client

scope = "user-read-currently-playing user-modify-playback-state"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

# Get what the user is listening now
result = sp.current_user_playing_track()

#se for uma faixa
if result and result['context']['type'] == 'track':    

    #salva a ouvida no banco
    pass


In [26]:

from requests import post


with post('https://fiapfunctionapp.azurewebsites.net/api/predict_best_next_songs') as response:
    print(response.content)
    for song_id in response.json():
        sp.add_to_queue(f'spotify:track:{song_id}')


b''


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [21]:
for song_id in [
	"7oAuqs6akGnPU3Tb00ZmyM",
	"5Gf4tg1WJOzIw5iiOLBJzF",
	"7KA4W4McWYRpgf0fWsJZWB",
	"5ZmzzBiEAIVCkgsPyh80gt",
	"1QTSmXYIKq5jyz4iYKoxb0"
]:
    sp.add_to_queue(f'spotify:track:{song_id}')

In [15]:
# adiciona musica a fila 
sp.add_to_queue('spotify:track:1UmmSKPdCtBuJsmbZG9G3u')


In [10]:
import json
print(json.dumps(sp.auth_manager.get_access_token()))

{"access_token": "BQAN5nPk92h-uEjmkyl9oDD4yJWSF37rxI_Vf4hHliL6XY-r38T5-l-_DDcanPsSOlSAnr4igoiE_EMpGqkaCg3Nxq4pDmA67adzpRDKwHfHW9c6xCA3FlVuckQxsICG1lK_5RCHZWtbiovb_xJ9zk5SJ91CST0a2tze3WQYtiSjh-Hv7riYyyxSlGhIVxRZT7iOUZ4L-Rc", "token_type": "Bearer", "expires_in": 3600, "refresh_token": "AQBDy4pIh5mN7cucwY44t7QtMsdtS2magBqL5DimFjDRzpq6VVL3dPDzLTxNcuu207CD1VBTlCDHr92HggdjbbQM3oxU1_VZVmSEU8JY476DqklsCymxgBmvE9TtVv2pxIs", "scope": "user-read-currently-playing", "expires_at": 1684805866}


C:\Users\netoc\AppData\Local\Temp\ipykernel_23440\4187654491.py:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  print(json.dumps(sp.auth_manager.get_access_token()))


In [ ]:
from requests import post
chunk_size=50
for chunk in [[d|{'album':d['album']['id']} 
  |{'artists':d['artists'][0]['id']}
  |{'available_markets':''}
  |{'external_ids':''}
  |{'external_urls':''}
   for d in chunk] 
    for chunk in [song_ids[i:i + chunk_size] 
        for i in range(0, len(song_ids), chunk_size)]]:
    print(len(chunk))
    with post('https://fiapfunctionapp.azurewebsites.net/api/songs', json = chunk) as response:
        response.raise_for_status()

    



In [10]:
scope = "user-read-currently-playing playlist-modify-private"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
# Get what the user is listening now
result = sp.current_user_playing_track()

SpotifyOauthError: Server listening on localhost has not been accessed

In [7]:
result

NameError: name 'result' is not defined

chunk

In [ ]:
[[d|{'album':d['album']['id']} 
  |{'artists':d['artists'][0]['id']}
  |{'available_markets':''}
  |{'external_ids':''}
  |{'external_urls':''}
   for d in chunk] for chunk in [song_ids[i:i + chunk_size] for i in range(0, len(song_ids), chunk_size)]][0]

In [ ]:
 sp.current_user_saved_tracks(limit=limit, offset=offset)

In [ ]:
#pega as musicas ouvidas
import datetime


import spotipy
from spotipy.oauth2 import SpotifyOAuth
from tqdm import tqdm

# Set up the Spotify API client
scope = "user-read-recently-played"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

# Get the saved tracks from the user's library
offset = 0
limit = 50  # Maximum number of tracks to retrieve in one request
listens  = []

# pega o timestam atual
current_timestamp = int(datetime.datetime.now().timestamp() * 1000)

blocks_before = 24*(31+28+31+30+16)
#blocos de 1hr, em milisegundos
steps = 1000*60*60

#lista de blocos de milisegundos
befores = [current_timestamp - (steps *( i)) for i in range(blocks_before)]

for timestamp_before in befores:

    results = sp.current_user_recently_played(limit=limit, before=timestamp_before)
    tracks = results["items"]

    listens.extend(results["items"])
    
    os.system('cls')
    print(results)



In [ ]:


listens

In [ ]:
filtered_listens= [{k: v for k,v in l.items() if k in [ 'played_at', 'context']}|{'track_id':l['track']['id']} for l in listens]
import pandas as pd
pd.DataFrame(filtered_listens).to_csv('Cayro_filtered_listens_20230516.csv',index=False)

In [ ]:
len(filtered_listens)

In [ ]:
befores[-1]

In [ ]:
musics = list(set([f['track']['uri'] for f in listens ]))
musics


In [ ]:
filtered_listens


In [ ]:
(current_timestamp-1684369678768)/1000/60

In [ ]:
1684369678768